# 推文清洗

In [7]:
import pandas as pd
import numpy as np
import os
import sys
import nltk
from nltk import word_tokenize
import time
from IPython.display import clear_output as clear

In [15]:
#"F:\Russia_Ukraine_Tweets\3.1-3.31(finished)"
dates = []
for filepath, dirnames, filenames in os.walk(r'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)'):
    for filename in filenames:
        date = os.path.join(filepath, filename)
        dates.append(date)

In [16]:
dates

['F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-22.txt_tweets.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-23.txt_tweets.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-24.txt_tweets.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_1.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_2.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_3.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_4.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_5.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_6.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_7.csv',
 'F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\\tweet_ids_day_2022-2-25.txt_tweets_8.csv',
 'F:/

In [10]:
import re
#写一个清洗函数

def clean_text(text):
    # 用空字符串替换文本中所有以"http"开头的连续字符串，并将文本转换为小写
    text = re.sub("http\S+", "", text).lower()
    # 使用列表推导式将文本拆分为单词列表，并过滤掉所有以@开头的单词
    text = " ".join([w for w in text.split() if w[0]!='@'])
    # 用空格替换文本中的所有非字母和数字字符，但不替换标点符号，并将剩余的文本转换为小写
    text = re.sub("[^a-zA-Z0-9\u4e00-\u9fa5,，.。？?！!;；:]+", " ", text).lower()
    return text

test_text = 'hello @Apple, https://apple.com 李John 12，34'
clean_text(text=test_text)


'hello 李john 12，34'

In [11]:

def word_more_than(text,num):#写一个检测text中单词数量是否大于num的函数
    # 检测一段文本中的英文单词
    tokens = nltk.word_tokenize(text)
    # 过滤掉标点符号
    words = [token for token in tokens if token.isalpha()]
    if len(words)>num:
        return 1
    else:
        return 0


In [12]:
dates[10][:-4]

'F:/Russia_Ukraine_Tweets/3.1-3.31(finished)/tweet_ids_day_2022-3-11.txt_tweets_1'

In [17]:
for date in dates:
    name = date[:-4] + '_clean.csv'
    if os.path.exists(name):#如果已经洗过了，就跳过
        clear(wait=True)
        print(name,' generated', flush = True)
        continue
        
    else:
        print(name,' generating...')
        data = pd.read_csv(date, index_col=0, dtype={'Tweet_place':str, 'Tweet_hashtags':str, 'Username':str })
        data = data[data['Tweet_language'] == 'en']#过滤英文推特
        data = data.reset_index(drop=True)
        data['Tweet_isRT']=0
        for i in range(len(data)):
            if data.loc[i].Tweet_content[:2]=='RT':
                data.loc[i, 'Tweet_isRT'] = 1
                data.loc[i, 'Tweet_content'] = data.loc[i, 'Tweet_content'][3:]#删除文本中的RT标识

        data['Tweet_content'] = data['Tweet_content'].apply(clean_text)#去除url、@
        data.dropna(subset=['Tweet_content'])

        #删除小于等于3个单词的推文
        for index, row in data.iterrows():
            text = row['Tweet_content']
            if not word_more_than(text, 2):
                data.drop(index, inplace=True)
                
        clear(wait=True)
        print(name,' generated', flush = True)
        data.to_csv(name)

F:/Russia_Ukraine_Tweets/2.22-2.28(finished)\tweet_ids_day_2022-2-28.txt_tweets_7_clean.csv  generated
